### F Score Implementation - May 2019 

In [1]:
# importing packages 
from datetime import datetime
import locale
from locale import atof
import pickle
import time
import random
import re

import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# sqlalchemy packages 
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, Numeric, String, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship, backref
from sqlalchemy.ext.declarative import declarative_base 

#### Creating Universe for F Score

In [33]:
# Getting DATA of KOSPI 
def get_univ_data(market, page_num):
    def clean_value(value):
        if ',' in value: 
            value = value.replace(',', '')
    
        return value
    
    mkt = market 
    
    if mkt == 0: 
        kospi_id_ls = []
        kospi_name_ls = []
        kospi_number_classes = []
    
        for page in range(1, page_num+1):
            url = """http://finance.naver.com/sise/field_submit.nhn?menu=market_sum&returnUrl=http%3A%2F%2Ffinance.naver.com%2Fsise%2Fsise_market_sum.nhn%3Fsosok%3D{0}%26page%3D{1}&fieldIds={2}&fieldIds={3}&fieldIds={4}&fieldIds={5}&fieldIds={6}&fieldIds={7}""".format(mkt, page, 'quant', 'listed_stock_cnt', 'market_sum', 'property_total', 'debt_total', 'pbr')
    
            req = requests.get(url)
            html = req.text 
    
            soup = BeautifulSoup(html, 'lxml')
        
            kospi_ids = soup.select('a[href*="/item/main.nhn"]')
            kospi_univ_sets = soup.select('td[class^="number"]')
        
            # Looping stock ids and names 
            for id in kospi_ids: 
                kospi_id_ls.append(str(id).strip('"<>/a').split('=')[-1].split('">')[0])
                kospi_name_ls.append(str(id).strip('"<>/a').split('=')[-1].split('">')[1])
        
            # Looping Number Classes for each stock
            for item in kospi_univ_sets: 
                kospi_number_classes.append(item.string)
                
        # Convert stock ids & names lists to Pandas DF 
        df_kospi = pd.DataFrame({'id' : kospi_id_ls, 'name': kospi_name_ls, 'market_type': 1}, columns = ['id', 'name','market_type', 'quant', 'market_sum', 'property_total', 'debt_total', 'listed_stock_cnt', 'pbr', 'face_value',])

        # Slicing and creating an individual list for each category 
        present_value = [clean_value(v) for v in kospi_number_classes[::10]]
        face_value = [clean_value(v) for v in kospi_number_classes[3::10]]
        quant = [clean_value(v) for v in kospi_number_classes[4::10]]
        listed_stock_cnt = [clean_value(v) for v in kospi_number_classes[5::10]]
        market_sum = [clean_value(v) for v in kospi_number_classes[6::10]] 
        property_total = [clean_value(v) for v in kospi_number_classes[7::10]]
        debt_total = [clean_value(v) for v in kospi_number_classes[8::10]]
        pbr = [clean_value(v) for v in kospi_number_classes[9::10]] 

        # Put each category list to DF
        df_kospi['present_value'] = pd.to_numeric(present_value, errors='coerce')
        df_kospi['quant'] = pd.to_numeric(quant, errors='coerce')
        df_kospi['property_total'] = pd.to_numeric(property_total, errors='coerce')
        df_kospi['debt_total'] = pd.to_numeric(debt_total, errors='coerce')
        df_kospi['listed_stock_cnt'] = pd.to_numeric(listed_stock_cnt, errors='coerce')
        df_kospi['pbr'] = pd.to_numeric(pbr, errors='coerce')
        df_kospi['market_sum'] = pd.to_numeric(market_sum, errors='coerce')
        df_kospi['face_value'] = pd.to_numeric(face_value, errors='coerce')   
        
        return df_kospi
    
    elif mkt == 1: 
        kosdaq_id_ls = []
        kosdaq_name_ls = [] 
        kosdaq_number_classes = []
        
        for page in range(1, page_num+1):
            url = """http://finance.naver.com/sise/field_submit.nhn?menu=market_sum&returnUrl=http%3A%2F%2Ffinance.naver.com%2Fsise%2Fsise_market_sum.nhn%3Fsosok%3D{0}%26page%3D{1}&fieldIds={2}&fieldIds={3}&fieldIds={4}&fieldIds={5}&fieldIds={6}&fieldIds={7}""".format(mkt, page, 'quant', 'listed_stock_cnt', 'market_sum', 'property_total', 'debt_total', 'pbr' )
            
            req = requests.get(url)
            html = req.text
            
            soup = BeautifulSoup(html, 'lxml')
            
            kosdaq_ids = soup.select('a[href*="/item/main.nhn"]')
            kosdaq_univ_sets = soup.select('td[class^="number"]')
    
            # Looping KOSDAQ ids and names 
            for id in kosdaq_ids:
                kosdaq_id_ls.append(str(id).strip('"<>/a').split('=')[-1].split('">')[0])
                kosdaq_name_ls.append(str(id).strip('"<>/a').split('=')[-1].split('">')[1])
    
            # Looping Number classes for each stock 
            for item in kosdaq_univ_sets: 
                kosdaq_number_classes.append(item.string)
       
        # convert KOSDAQ ids & names lists to Pandas DF 
        df_kosdaq = pd.DataFrame({'id' : kosdaq_id_ls, 'name' : kosdaq_name_ls, 'market_type' : 2}, columns = ['id', 'name','market_type', 'quant', 'market_sum', 'property_total', 'debt_total', 'listed_stock_cnt', 'pbr', 'face_value',])
        
        # Slicing and Put into df_kosdaq
        # Slicing and creating an individual list for each category 
        present_value = [clean_value(v) for v in kosdaq_number_classes[::10]]
        face_value = [clean_value(v) for v in kosdaq_number_classes[3::10]]
        quant = [clean_value(v) for v in kosdaq_number_classes[4::10]]
        listed_stock_cnt = [clean_value(v) for v in kosdaq_number_classes[5::10]]
        market_sum = [clean_value(v) for v in kosdaq_number_classes[6::10]]
        property_total = [clean_value(v) for v in kosdaq_number_classes[7::10]]
        debt_total = [clean_value(v) for v in kosdaq_number_classes[8::10]]
        pbr = [clean_value(v) for v in kosdaq_number_classes[9::10]] 

        # Put each category list to DF
        df_kosdaq['present_value'] = pd.to_numeric(present_value, errors='coerce')
        df_kosdaq['quant'] = pd.to_numeric(quant, errors='coerce')
        df_kosdaq['property_total'] = pd.to_numeric(property_total, errors='coerce')
        df_kosdaq['debt_total'] = pd.to_numeric(debt_total, errors='coerce')
        df_kosdaq['listed_stock_cnt'] = pd.to_numeric(listed_stock_cnt, errors='coerce')
        df_kosdaq['pbr'] = pd.to_numeric(pbr, errors='coerce')
        df_kosdaq['market_sum'] = pd.to_numeric(market_sum, errors='coerce')
        df_kosdaq['face_value'] = pd.to_numeric(face_value, errors='coerce')    
        
        return df_kosdaq

In [34]:
df_kospi = get_univ_data(0, 31) #2019년 5월 기준 31페이지
df_kosdaq = get_univ_data(1, 27) #2019년 5월 기준 27 페이지

In [8]:
df_univ = pd.concat([df_kospi, df_kosdaq], ignore_index=True)

In [9]:
bm_columns = ['id', 'market_type', 'market_sum', 'property_total', 'debt_total', 'pbr']
df_bm = df_univ[bm_columns]

In [10]:
df_bm.head(5)

,id,market_type,market_sum,property_total,debt_total,pbr
0,005930,1,2507309,3393572.0,916041.0,1.19
1,000660,1,516154,636583.0,168060.0,1.04
2,005935,1,277313,NaN,NaN,0.95
3,005380,1,275632,1806558.0,1067597.0,0.50
4,068270,1,241259,35406.0,9078.0,9.47


In [11]:
df_bm['book_value'] = df_bm['property_total'] - df_bm['debt_total']

/Users/daesikkim/anaconda3/envs/blowfish3/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
df_bm['bm_ratio'] = df_bm['book_value'] / df_bm['market_sum']

/Users/daesikkim/anaconda3/envs/blowfish3/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
df_bm['bm_decile'] = pd.qcut(df_bm['bm_ratio'], 10, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
df_bm['pbr_decile'] = pd.qcut(df_bm['pbr'], 10, labels=[10, 9, 8, 7, 6, 5, 4, 3, 2, 1])

/Users/daesikkim/anaconda3/envs/blowfish3/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/daesikkim/anaconda3/envs/blowfish3/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [14]:
df_bm.head(10)

,id,market_type,market_sum,property_total,debt_total,pbr,book_value,bm_ratio,bm_decile,pbr_decile
0,005930,1,2507309,3393572.0,916041.0,1.19,2477531.0,0.988124,6,5
1,000660,1,516154,636583.0,168060.0,1.04,468523.0,0.907719,6,6
2,005935,1,277313,NaN,NaN,0.95,NaN,NaN,NaN,6
3,005380,1,275632,1806558.0,1067597.0,0.50,738961.0,2.680970,10,9
4,068270,1,241259,35406.0,9078.0,9.47,26328.0,0.109128,1,1
5,051910,1,232955,289441.0,116220.0,1.48,173221.0,0.743581,5,4
6,055550,1,215287,4596005.0,4229491.0,0.60,366514.0,1.702444,9,8
7,017670,1,210343,423691.0,200199.0,0.83,223492.0,1.062512,7,7
8,012330,1,206816,430711.0,123677.0,0.67,307034.0,1.484576,8,8
9,005490,1,204889,782483.0,314887.0,0.43,467596.0,2.282192,10,10


In [15]:
# Fscore university top 50%
df_fscore_univ = df_bm.query('bm_decile > 5')
df_fscore_univ_ids = df_fscore_univ['id']

In [16]:
len(df_fscore_univ_ids)

1055

In [17]:
df_fscore_univ_20 = df_bm.query('bm_decile > 8')
df_ids_20 = df_fscore_univ_20['id']

In [18]:
len(df_ids_20)

422

### BM Ratio by markets 

In [74]:
df_kospi.keys()

Index(['id', 'name', 'market_type', 'quant', 'market_sum', 'property_total',
       'debt_total', 'listed_stock_cnt', 'pbr', 'face_value', 'present_value'],
      dtype='object')

In [75]:
df_bm_kospi = df_kospi.copy()

In [76]:
df_bm_kospi['book_value'] = df_bm_kospi.property_total - df_bm_kospi.debt_total

In [77]:
df_bm_kospi['bm_ratio'] = df_bm_kospi.book_value / df_bm_kospi.market_sum 

In [78]:
df_kosdaq.keys()

Index(['id', 'name', 'market_type', 'quant', 'market_sum', 'property_total',
       'debt_total', 'listed_stock_cnt', 'pbr', 'face_value', 'present_value'],
      dtype='object')

In [79]:
df_bm_kosdaq = df_kosdaq.copy()

In [80]:
df_bm_kosdaq['book_value'] = df_bm_kosdaq.property_total - df_bm_kosdaq.debt_total

In [81]:
df_bm_kosdaq['bm_ratio'] = df_bm_kosdaq.book_value / df_bm_kosdaq.market_sum

In [82]:
df_bm_kospi['bm_decile'] = pd.qcut(df_bm_kospi['bm_ratio'], 10, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
df_bm_kospi['pbr_decile'] = pd.qcut(df_bm_kospi['pbr'], 10, labels=[10, 9, 8, 7, 6, 5, 4, 3, 2, 1])

df_bm_kosdaq['bm_decile'] = pd.qcut(df_bm_kosdaq['bm_ratio'], 10, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
df_bm_kosdaq['pbr_decile'] = pd.qcut(df_bm_kosdaq['pbr'], 10, labels=[10, 9, 8, 7, 6, 5, 4, 3, 2, 1])

In [93]:
df_bm_kospi_t20 = df_bm_kospi.query('(bm_decile >= 8) & (pbr_decile <= 8)')
df_bm_kosdaq_t20 = df_bm_kosdaq.query('(bm_decile >= 8) & (pbr_decile <= 8)')

In [94]:
df_bm_t20 = pd.concat([df_bm_kospi_t20, df_bm_kosdaq_t20])

In [95]:
df_bm_kospi_t50 = df_bm_kospi.query('(bm_decile >= 5) & (pbr_decile <= 5)')
df_bm_kosdaq_t50 = df_bm_kosdaq.query('(bm_decile >= 5) & (pbr_decile <= 5)')

In [96]:
df_bm_t50 = pd.concat([df_bm_kospi_t50, df_bm_kosdaq_t50])

In [97]:
df_fscore_univ_ids = df_bm_t50['id']

In [100]:
len(df_fscore_univ_ids)

1223

In [110]:
df_bm = pd.concat([df_bm_kospi, df_bm_kosdaq])

In [111]:
df_bm

,id,name,market_type,quant,market_sum,property_total,debt_total,listed_stock_cnt,pbr,face_value,present_value,book_value,bm_ratio,bm_decile,pbr_decile
0,005930,삼성전자,1,15691950,2593871,3393572.0,916041.0,5969783,1.23,100,43450,2477531.0,0.955148,4,3
1,000660,SK하이닉스,1,3216142,515426,636583.0,168060.0,728002,1.03,5000,70800,468523.0,0.909001,4,4
2,005935,삼성전자우,1,1259235,288010,NaN,NaN,822887,0.99,100,35000,NaN,NaN,NaN,4
3,005380,현대차,1,534447,271359,1806558.0,1067597.0,213668,0.50,5000,127000,738961.0,2.723186,9,8
4,051910,LG화학,1,106674,233661,289441.0,116220.0,70592,1.48,5000,331000,173221.0,0.741335,3,3
5,068270,셀트리온,1,900924,233559,35406.0,9078.0,128329,9.16,1000,182000,26328.0,0.112725,1,1
6,055550,신한지주,1,736372,216709,4596005.0,4229491.0,474200,0.60,5000,45700,366514.0,1.691273,7,7
7,017670,SK텔레콤,1,79736,207516,423691.0,200199.0,80746,0.82,500,257000,223492.0,1.076987,5,5
8,005490,POSCO,1,167535,204017,782483.0,314887.0,87187,0.43,5000,234000,467596.0,2.291946,9,9
9,012330,현대모비스,1,109087,202527,430711.0,123677.0,95307,0.66,5000,212500,307034.0,1.516015,6,6


In [109]:
df_bm_t50

,id,name,market_type,quant,market_sum,property_total,debt_total,listed_stock_cnt,pbr,face_value,present_value,book_value,bm_ratio,bm_decile,pbr_decile
3,005380,현대차,1,534447,271359,1806558.0,1067597.0,213668,0.50,5000,127000,738961.0,2.723186,9,8
6,055550,신한지주,1,736372,216709,4596005.0,4229491.0,474200,0.60,5000,45700,366514.0,1.691273,7,7
7,017670,SK텔레콤,1,79736,207516,423691.0,200199.0,80746,0.82,500,257000,223492.0,1.076987,5,5
8,005490,POSCO,1,167535,204017,782483.0,314887.0,87187,0.43,5000,234000,467596.0,2.291946,9,9
9,012330,현대모비스,1,109087,202527,430711.0,123677.0,95307,0.66,5000,212500,307034.0,1.516015,6,6
12,105560,KB금융,1,503245,194631,4795883.0,4438753.0,418112,0.52,5000,46550,357130.0,1.834908,7,8
14,028260,삼성물산,1,283410,181533,424067.0,198577.0,189690,0.79,100,95700,225490.0,1.242143,5,5
15,034730,SK,1,50496,169568,1194570.0,685691.0,70360,0.82,200,241000,508879.0,3.001032,10,5
16,000270,기아차,1,948059,166807,517866.0,245431.0,405363,0.61,5000,41150,272435.0,1.633235,7,7
17,032830,삼성생명,1,110390,162000,2894277.0,2589222.0,200000,0.50,500,81000,305055.0,1.883056,8,8


#### Crawl BalanceSheet, IncomeStatement, CashFlow

In [63]:
# Headless Chrome
options = webdriver.ChromeOptions()
#options.add_argument('headless')
#options.add_argument('window-size=1920x1080')
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome')

In [112]:
# Start Chrome with Selenium
driver = webdriver.Chrome('/Users/daesikkim/Downloads/chromedriver', chrome_options=options) # chrome_options=options
driver.implicitly_wait(3)

/Users/daesikkim/anaconda3/envs/blowfish3/lib/python3.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [102]:
# A function to crawl the industry classification, Market & ICS 
def crawl_ics(code): 
    #pArea > div.wrapper-table > div > table > tbody > tr:nth-child(1) > td > dl > dt:nth-child(3)
    # getting the page source and making the soup! 
    html_ics = driver.page_source 
    soup_ics = BeautifulSoup(html_ics, 'lxml')
    
    # getting valus for ICS 
    m_ics = soup_ics.select('#pArea > div.wrapper-table > div > table > tbody > tr:nth-of-type(1) > td > dl > dt:nth-of-type(3)')
    w_ics = soup_ics.select('#pArea > div.wrapper-table > div > table > tbody > tr:nth-of-type(1) > td > dl > dt:nth-of-type(4)')
    
    m_ics = m_ics[0].string.split(' : ')[-1]
    w_ics = w_ics[0].string.split(' : ')[-1]
    
    ics = {'m_ics': [m_ics]*5, 'w_ics': [w_ics]*5}
    
    return ics

#### 재무용어 및 영어정리
***
__재무재표__
- total asset = 자산총계
- long term debt = long term liabilities = 비유동부채
- long term borrowing = 장기차입금
- current asset = 유동자산
- current liabilities = 유동부채
- shareholder equity = 자본총계
- stock issued = 발행주식수
- intangible asset = 무형자산
- book value 1 = 순자산1
- book value 2 = 순자산2
- total liabilities = 부채총계
***

In [103]:
def crawl_balsheet(code):
    
    # click Bal Sheet Tab 
    balsheet_tab = driver.find_element_by_css_selector("#rpt_tab2")
    balsheet_tab.click()
    time.sleep(0.8)
    
    # getting the page source and making the soup! 
    html_balsheet = driver.page_source
    soup_balsheet = BeautifulSoup(html_balsheet, 'lxml')
    
    r = re.compile(r'<div id="([0-9a-zA-Z]{12})">')
    div_id = "#" + r.findall(html_balsheet)[0]

    # getting values from "Balance Sheet" 
    total_asset = soup_balsheet.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(1) > td.num')
    lt_debt = soup_balsheet.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(175) > td.num')
    lt_borrowing = soup_balsheet.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(181) > td.num')
    current_asset = soup_balsheet.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(2) > td.num')
    current_liabilities = soup_balsheet.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(128) > td.num')
    shareholder_equity = soup_balsheet.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(216) > td.num')
    stock_issued = soup_balsheet.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(249) > td.num')
    intangible_asset = soup_balsheet.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(73) > td.num')
    book_value1 = soup_balsheet.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(125) > td.num')
    book_value2 = soup_balsheet.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(126) > td.num')
    total_liabilities = soup_balsheet.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(127) > td.num')
    
    # convert string --> float 
    balsheet_vals = list(map(lambda l: [float(i.string.replace(',','')) if i.string != u'\xa0' else float(i.string.replace(u'\xa0','0')) for i in l[0:5]], [total_asset, lt_debt, lt_borrowing, current_asset, current_liabilities, shareholder_equity, stock_issued, intangible_asset, book_value1, book_value2, total_liabilities]))
    
    balsheet_dict = {'total_asset': balsheet_vals[0],
                     'lt_debt': balsheet_vals[1],
                     'lt_borrowing': balsheet_vals[2],
                     'current_asset': balsheet_vals[3],
                     'current_liabilities': balsheet_vals[4],
                     'shareholder_equity': balsheet_vals[5],
                     'stock_issued' : balsheet_vals[6],
                     'intangible_asset' : balsheet_vals[7], 
                     'book_value1' : balsheet_vals[8], 
                     'book_value2' : balsheet_vals[9], 
                     'total_liabilities' : balsheet_vals[10]}
    
    return balsheet_dict 

***
__포괄손익계산서__
- operating income = 영업이익
- extra income = 기타영업외손익
- total sales = 매출액
- gross profit = 매출총이익
- cogs = 매출원가
***

In [104]:
# A function to crawl an income statement 
def crawl_incomestate(code):
    
    # click Income Statement Tab 
    incomestate_tab = driver.find_element_by_css_selector("#rpt_tab1")
    incomestate_tab.click()
    time.sleep(0.8)
    
    # getting the page source and making the soup
    html_incomestate = driver.page_source
    soup_incomestate = BeautifulSoup(html_incomestate, 'lxml') 
    
    r = re.compile(r'<div id="([0-9a-zA-Z]{12})">')
    div_id = "#" + r.findall(html_incomestate)[0]
    
    # getting values from "Income Statement" 
    operating_income = soup_incomestate.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(58) > td.num')
    extra_income = soup_incomestate.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(150) > td.num')
    total_sales = soup_incomestate.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(1) > td.num')
    gross_profit = soup_incomestate.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(26) > td.num')
    cogs = soup_incomestate.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(15) > td.num')
    
    # convert string --> float 
    incomestate_vals = list(map(lambda l: [float(i.string.replace(',','')) if i.string != u'\xa0' else float(i.string.replace(u'\xa0','0')) for i in l[0:5]], [operating_income, extra_income, total_sales, gross_profit, cogs]))
    
    incomestate_dict = {'operating_income': incomestate_vals[0],
                        'extra_income': incomestate_vals[1],
                        'total_sales': incomestate_vals[2],
                        'gross_profit': incomestate_vals[3],
                        'cogs': incomestate_vals[4]}
    
    return incomestate_dict

***
__현금흐름표__
- cashflow operation = 영업활동으로인한 현금흐름
***

In [105]:
# A function to crawl a cash flow 
def crawl_cashflow(code):
    
    # click Cash Flow tab 
    cashflow_tab = driver.find_element_by_css_selector("#rpt_tab3")
    cashflow_tab.click
    time.sleep(0.8)
    
    # getting the page source and making the soup 
    html_cashflow = driver.page_source 
    soup_cashflow = BeautifulSoup(html_cashflow, 'lxml')
    
    r = re.compile(r'<div id="([0-9a-zA-Z]{12})">')
    div_id = "#" + r.findall(html_cashflow)[0]
    
    # getting values from "Income Statement" 
    cf_operation = soup_cashflow.select(f'{div_id}' + ' > table:nth-of-type(2) > tbody > tr:nth-of-type(1) > td.num')
    
    # convert string --> float 
    cashflow_vals = list(map(lambda l: [float(i.string.replace(',','')) if i.string != u'\xa0' else float(i.string.replace(u'\xa0', '0')) for i in l[0:5]], [cf_operation]))
    
    cashflow_dict = {'cf_operation': cashflow_vals[0]}
    
    return cashflow_dict

In [106]:
# A function to transform a list to DF 
def merge_n_convert(code, year, fs_dict): 
    dict_individual = {'code': [code]*5,
                       'year': year}
    
    dict_individual.update(fs_dict)
    
    df_individual = pd.DataFrame(dict_individual, columns = ('code', 'm_ics', 'w_ics', 'year', \
                                    'total_asset', 'lt_debt', 'lt_borrowing', 'current_asset', 'current_liabilities', 'shareholder_equity', 'stock_issued', 'intangible_asset', \
                                    'operating_income', 'extra_income', 'total_sales', 'gross_profit', 'cogs', \
                                    'cf_operation'))
    
    return df_individual                              

In [107]:
# Creating f score DF 
year = ['2014', '2015', '2016', '2017', '2018']
fs_dict = {}
df_fbase = pd.DataFrame(columns = ('code', 'm_ics', 'w_ics', 'year', \
                                    'total_asset', 'lt_debt', 'lt_borrowing', 'current_asset', 'current_liabilities', 'shareholder_equity', 'stock_issued', 'intangible_asset', \
                                    'operating_income', 'extra_income', 'total_sales', 'gross_profit', 'cogs', \
                                    'cf_operation'))
df_fcalc = pd.DataFrame(columns = ('code', 'year', 'cal_roa', 'cal_cfo', 'delta_roa', 'accrual', 'delta_lever', 'delta_liquid', 'eq_offer', 'delta_margin', 'delta_turn'))
df_fscore = pd.DataFrame(columns = ('code', 'year','f_roa', 'f_cfo', 'f_droa', 'f_accrual', 'f_dlever', 'f_dliquid', 'f_equityoffer', 'f_dmargin', 'f_dturnover', 'f_total'))

#### Crawling codes below

In [113]:
# Do crawl!
count = 0
init_time = time.time()
for i in df_fscore_univ_ids:
    start_time = time.time()
     # load the page! 
    driver.get("http://finance.naver.com/item/coinfo.nhn?code={}".format(i))
    time.sleep(random.randrange(2, 8, 1))
    
    # move to the relavent frame
    frame = WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "#coinfo_cp")))
    #driver.switch_to.frame(frame)

    # select to the financial statement tab
    finstate_tab = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#header-menu > div.wrapper-menu > dl > dt:nth-of-type(3) > a")))
    finstate_tab.click()
    
    time.sleep(0.8)
    print(f"Company {i} processing")
    
    code = i
    for d in [crawl_ics(i), crawl_balsheet(i), crawl_incomestate(i), crawl_cashflow(i)]:
        fs_dict.update(d)                
    
    try:
        df_company = merge_n_convert(i, year, fs_dict)

        df_fbase = pd.concat([df_fbase, df_company], ignore_index=True)
        count += 1
    except: 
        count += 1
        continue

    #print(df_fbase)
    elapsed = time.time() - start_time
    progress = count/len(df_fscore_univ_ids)*100
    print(f"{count} / {len(df_fscore_univ_ids)} >>> {progress}% Processed!!!")
    print(time.gmtime(elapsed))

Company 005380 processing
1 / 1223 >>> 0.08176614881439084% Processed!!!
time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=26, tm_wday=3, tm_yday=1, tm_isdst=0)
Company 055550 processing
Company 017670 processing
3 / 1223 >>> 0.24529844644317253% Processed!!!
time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=25, tm_wday=3, tm_yday=1, tm_isdst=0)
Company 005490 processing
4 / 1223 >>> 0.3270645952575634% Processed!!!
time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=23, tm_wday=3, tm_yday=1, tm_isdst=0)
Company 012330 processing
5 / 1223 >>> 0.4088307440719542% Processed!!!
time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=20, tm_wday=3, tm_yday=1, tm_isdst=0)
Company 105560 processing
Company 028260 processing
7 / 1223 >>> 0.5723630417007358% Processed!!!
time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=24, tm_wday=3, tm_yday=1, tm_isdst

In [ ]:
df_fbase

#### Save & Load

In [36]:
with open('fscore_may2019.txt', 'wb') as f:
    pickle.dump(df_fbase, f)

In [2]:
with open('fscore_may2019.txt', 'rb') as f:
    df_fbase = pickle.load(f)

In [4]:
df_fbase.head(10)

,code,m_ics,w_ics,year,total_asset,lt_debt,lt_borrowing,current_asset,current_liabilities,shareholder_equity,stock_issued,intangible_asset,operating_income,extra_income,total_sales,gross_profit,cogs,cf_operation
0,005930,전기전자,반도체와반도체장비,2014,2304229.6,103208.6,239.9,1151460.3,520139.1,1680881.9,170133.0,47854.7,250250.7,15416.2,2062059.9,779271.9,1282788.0,2062059.9
1,005930,전기전자,반도체와반도체장비,2015,2421795.2,126168.1,1936.0,1248147.3,505029.1,1790598.1,170133.0,53963.1,264134.4,-20374.9,2006534.8,771713.6,1234821.2,2006534.8
2,005930,전기전자,반도체와반도체장비,2016,2621743.2,145072.0,11791.1,1414297.0,547041.0,1929630.3,161193.0,53440.2,292406.7,7744.5,2018667.5,815890.3,1202777.2,2018667.5
3,005930,전기전자,반도체와반도체장비,2017,3017520.9,200855.5,17569.1,1469824.6,671751.1,2144914.3,147349.0,147604.8,536450.4,15910.1,2395753.8,1102847.2,1292906.6,2395753.8
4,005930,전기전자,반도체와반도체장비,2018,3393572.4,225225.6,349.6,1746974.2,690815.1,2477531.8,6792669.0,148916.0,588866.7,3430.2,2437714.2,1113770.0,1323944.1,2437714.2
5,000660,전기전자,반도체와반도체장비,2014,268832.8,30816.7,12627.7,103635.1,57653.0,180363.0,728002.0,13366.8,51094.7,362.6,171255.7,76638.4,94617.3,171255.7
6,000660,전기전자,반도체와반도체장비,2015,296779.1,34495.1,15120.0,97600.3,48407.0,213877.0,728002.0,17049.0,53361.0,-1084.6,187980.0,82826.5,105153.5,187980.0
7,000660,전기전자,반도체와반도체장비,2016,322160.3,40316.5,20957.4,98389.8,41608.5,240235.3,728002.0,19155.9,32767.5,-516.1,171979.8,64108.4,107871.4,171979.8
8,000660,전기전자,반도체와반도체장비,2017,454184.6,34814.1,20803.3,173104.4,81161.3,338209.2,728002.0,22472.9,137213.3,-409.8,301094.3,174075.9,127018.4,301094.3
9,000660,전기전자,반도체와반도체장비,2018,636583.4,37741.5,21615.7,198941.5,130318.5,468523.3,728002.0,26787.7,208437.5,-655.5,404450.7,252642.3,151808.4,404450.7


In [6]:
print(len(df_fbase))
print(len(df_fbase)/5)

4865
973.0


#### F Score Calculation

In [8]:
df_fbase.keys()

Index(['code', 'm_ics', 'w_ics', 'year', 'total_asset', 'lt_debt',
       'lt_borrowing', 'current_asset', 'current_liabilities',
       'shareholder_equity', 'stock_issued', 'intangible_asset',
       'operating_income', 'extra_income', 'total_sales', 'gross_profit',
       'cogs', 'cf_operation'],
      dtype='object')

In [9]:
df_fs_01 = df_fbase[['code', 'year', 'total_asset', 'operating_income', 
                    'cf_operation', 'lt_debt', 'lt_borrowing', 
                    'current_asset', 'current_liabilities', 'stock_issued', 
                    'gross_profit', 'total_sales']]

In [11]:
# Rearranging columns - lagged total asset ("total assets fromt he beginning of the year")
df_fs_01['l_total_asset'] = df_fs_01.loc[:, 'total_asset'].shift(1)
df_fs_01['l_total_asset'] = df_fs_01.loc[:, 'l_total_asset'].where(df_fs_01.loc[:, 'year'] != '2014') # converting to NaN for 2014


/Users/daesikkim/anaconda3/envs/blowfish3/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/daesikkim/anaconda3/envs/blowfish3/lib/python3.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [12]:
# Average total assets = "rolling"
df_fs_01['avg_total_asset'] = df_fs_01.loc[:, 'total_asset'].where(df_fs_01.loc[:, 'year'] != '2014').rolling(2).mean()

/Users/daesikkim/anaconda3/envs/blowfish3/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [13]:
# Creating df_fs_02(calculation) and df_fs_03(fscore) dataframe
df_fs_02 = pd.DataFrame(columns = ('code', 'year', 'cal_roa', 
                                   'cal_cfo', 'delta_roa', 'accrual',
                                   'delta_lever', 'delta_liquid', 'eq_offer', 
                                   'delta_margin', 'delta_turn', 'gpa'))
df_fs_03 = pd.DataFrame(columns = ('code', 'year','f_roa', 
                                    'f_cfo', 'f_droa', 'f_accrual', 
                                    'f_dlever', 'f_dliquid', 'f_equityoffer', 
                                    'f_dmargin', 'f_dturnover', 'f_total', 'gpa_decile'))

In [15]:
# Calcuation
df_fs_02['code'] = df_fs_01['code']
df_fs_02['year'] = df_fs_01['year']

df_fs_02['cal_roa'] = df_fs_01['operating_income'] / df_fs_01['l_total_asset']
df_fs_02['cal_cfo'] = df_fs_01['cf_operation'] / df_fs_01['l_total_asset']
df_fs_02['delta_roa'] = df_fs_02['cal_roa'].diff()
df_fs_02['accurual'] = df_fs_02['cal_roa'] - df_fs_02['cal_cfo']

df_fs_02['lever'] = df_fs_01['lt_debt'] / df_fs_01['avg_total_asset']
df_fs_02['delta_lever'] = df_fs_02['lever'].diff()
df_fs_02['delta_liquid'] = df_fs_01['current_asset'].where(df_fs_01['year'] != '2014') / df_fs_01['current_liabilities'].where(df_fs_01['year'] != '2014')
df_fs_02['eq_offer'] = df_fs_01['stock_issued'].where(df_fs_01['year'] != '2014').diff()

df_fs_02['gmo'] = df_fs_01['gross_profit'].where(df_fs_01['year'] != '2014') / df_fs_01['total_sales'].where(df_fs_01['year'] != '2014')
df_fs_02['delta_margin'] = df_fs_02['gmo'].diff()
df_fs_02['atr'] = df_fs_01['total_sales'] / df_fs_01['l_total_asset']
df_fs_02['delta_turn'] = df_fs_02['atr'].diff()

df_fs_02['gpa'] = df_fs_01['gross_profit'] / df_fs_01['total_asset']

In [16]:
df_fs_02.head(10)

,code,year,cal_roa,cal_cfo,delta_roa,accrual,delta_lever,delta_liquid,eq_offer,delta_margin,delta_turn,gpa,accurual,lever,gmo,atr
0,005930,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.338192,NaN,NaN,NaN,NaN
1,005930,2015,0.114630,0.870805,NaN,NaN,NaN,2.471436,NaN,NaN,NaN,0.318654,-0.756175,NaN,0.384600,0.870805
2,005930,2016,0.120740,0.833542,0.006109,NaN,NaN,2.585358,-8940.0,0.019573,-0.037263,0.311201,-0.712802,0.057528,0.404173,0.833542
3,005930,2017,0.204616,0.913802,0.083876,NaN,0.013707,2.188049,-13844.0,0.056161,0.080260,0.365481,-0.709186,0.071235,0.460334,0.913802
4,005930,2018,0.195149,0.807853,-0.009467,NaN,-0.000973,2.528859,6645320.0,-0.003443,-0.105949,0.328200,-0.612704,0.070261,0.456891,0.807853
5,000660,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.285078,NaN,NaN,NaN,NaN
6,000660,2015,0.198491,0.699245,NaN,NaN,NaN,2.016244,NaN,NaN,NaN,0.279085,-0.500754,NaN,0.440613,0.699245
7,000660,2016,0.110410,0.579488,-0.088081,NaN,NaN,2.364656,0.0,-0.067846,-0.119757,0.198995,-0.469077,0.130276,0.372767,0.579488
8,000660,2017,0.425916,0.934610,0.315506,NaN,-0.040589,2.132844,0.0,0.205377,0.355123,0.383271,-0.508694,0.089687,0.578144,0.934610
9,000660,2018,0.458927,0.890498,0.033011,NaN,-0.020485,1.526579,0.0,0.046511,-0.044112,0.396872,-0.431572,0.069202,0.624655,0.890498


In [22]:
# converting df_fs_02 to df_fs_03
df_fs_03['code'] = df_fs_01['code']
df_fs_03['year'] = df_fs_01['year']

# ROA and its f-score in DataFrame
df_fs_03.f_roa = np.where(df_fs_02.cal_roa > 0, 1, 0)

# CFO and its f-score in DataFrame
df_fs_03.f_cfo = np.where(df_fs_02.cal_cfo > 0, 1, 0)

# delta ROA and its f-score in DataFrame 
df_fs_03.f_droa = np.where(df_fs_02.delta_roa > 0, 1, 0)

# Accrual and its f-score
df_fs_03.f_accrual = np.where(df_fs_02.accrual < 0, 1, 0)

# Delta_leverage and its f-score
df_fs_03.f_dlever = np.where(df_fs_02.delta_lever < 0, 1, 0)

# delta_liquidity and its f-score
df_fs_03.f_dliquid = np.where(df_fs_02.delta_liquid > 0, 1, 0)

# equity offer and its f-score
df_fs_03.f_equityoffer = np.where(df_fs_02.eq_offer > 0, 0, 1)

# Delta_margin and its f-score 
df_fs_03.f_dmargin = np.where(df_fs_02.delta_margin > 0, 1, 0)

# Delta_turnover and its f-score
df_fs_03.f_dturnover = np.where(df_fs_02.delta_turn > 0, 1, 0)

# total f_score 
df_fs_03.f_total = df_fs_03.f_roa + \
                   df_fs_03.f_cfo + \
                   df_fs_03.f_droa + \
                   df_fs_03.f_accrual + \
                   df_fs_03.f_dlever + \
                   df_fs_03.f_dliquid + \
                   df_fs_03.f_equityoffer + \
                   df_fs_03.f_dmargin + \
                   df_fs_03.f_dturnover 

# gpa decile
df_fs_03.gpa_decile = pd.qcut(df_fs_02['gpa'], 10, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])


In [23]:
df_fs_03.head()

,code,year,f_roa,f_cfo,f_droa,f_accrual,f_dlever,f_dliquid,f_equityoffer,f_dmargin,f_dturnover,f_total,gpa_decile
0,005930,2014,0,0,0,0,0,0,1,0,0,1,10
1,005930,2015,1,1,0,0,0,1,1,0,0,4,10
2,005930,2016,1,1,1,0,0,1,1,1,0,6,10
3,005930,2017,1,1,1,0,0,1,1,1,1,7,10
4,005930,2018,1,1,0,0,1,1,0,0,0,4,10


In [31]:
df_fs_2019 = df_fs_03.query('f_total >= 7')

In [32]:
len(df_fs_2019)

574